# 🧠 Gemma_RAG: Lightweight Retrieval-Augmented Generation with Gemma

A minimal example of using Retrieval-Augmented Generation (RAG) with Gemma models, integrated with `sentence-transformers`, `FAISS`, and `Streamlit`. This notebook is runnable on **free Colab instances**.

In [13]:
!pip install -Uq sentence-transformers transformers accelerate faiss-cpu timm

### 📦 Importing Required Libraries

This cell imports all the libraries needed for the project:
- `os` for accessing environment variables like HF tokens
- `torch` for deep learning with GPU support
- `transformers` for loading the Gemma language model
- `sentence-transformers` for creating semantic embeddings
- `faiss` for fast similarity search
- `numpy` for array manipulation and type casting

In [14]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

### 🧠 Model Setup and Token Loading

This cell loads the **authentication token**, sets the **model ID**, and initializes both the **tokenizer** and the **language model** (`Gemma-3n-E4B-it`) from the Hugging Face Hub. These steps are essential to prepare the model for inference (i.e., generating text).

- `token = os.environ.get("HF_TOKEN")`  
  Retrieves your Hugging Face token from environment variables. This is used to authenticate access to gated models (like Gemma-3n) securely. By storing the token in the environment, you avoid hardcoding sensitive info in your notebook.

- `model_id = "google/gemma-3n-E4B-it"`  
  Specifies the exact model you want to use from the Hugging Face Model Hub. In this case, you're using **Gemma-3n-E4B-it**, a 3-billion-parameter instruction-tuned language model developed by Google. This string acts as a reference for downloading both the tokenizer and model weights.

- `tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)`  
  Loads the tokenizer that matches the specified Gemma model. The tokenizer transforms raw input text (e.g., `"What happened?"`) into token IDs that the model understands. Using `AutoTokenizer` ensures the right tokenizer is chosen automatically based on the model’s config file. The `token=token` part ensures access to the tokenizer files from a private/gated model if necessary.

- `gemma_model = AutoModelForCausalLM.from_pretrained(model_id, token=token, torch_dtype=torch.bfloat16, device_map={"": 0})`  
  Loads the **Gemma-3n language model weights** for causal language modeling (i.e., left-to-right generation).  
  - `token=token`: Ensures authenticated access.  
  - `torch_dtype=torch.bfloat16`: Loads the model using Brain Float 16 precision, which is memory-efficient and optimized for newer GPUs like the A100.  
  - `device_map={"": 0}`: Places the full model on GPU 0 (i.e., `cuda:0`), preventing the runtime error you’d get if tensors are split across `cuda:0` and `cuda:1`.


In [15]:
token = os.environ.get("HF_TOKEN")
model_id = "google/gemma-3n-E4B-it"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
gemma_model = AutoModelForCausalLM.from_pretrained(model_id,token=token,torch_dtype=torch.bfloat16,device_map={"":0})

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.39s/it]


### 🔁 Creating the Text Generation Pipeline

This cell creates a **text generation pipeline** using Hugging Face’s `pipeline()` utility. The pipeline wraps the model and tokenizer together and handles the full process of generating natural language output from a prompt.

- `generator = pipeline("text-generation", ...)`  
  Initializes a high-level text generation pipeline for causal language models. This abstraction lets you input raw text and get full model-generated outputs without manually handling tokenization or decoding.

- `model=gemma_model`  
  Sets the pretrained Gemma model as the core component that will perform text generation.

- `tokenizer=tokenizer`  
  Supplies the tokenizer needed to convert input strings into token IDs that the model can understand.

- `device_map=0`  
  Assigns the model and data to GPU 0 (`cuda:0`). This is important to avoid device mismatch errors when using multiple GPUs.

- `torch_dtype=torch.bfloat16`  
  Sets the numerical precision for model weights and activations to bfloat16, which is memory-efficient and optimized for modern GPUs like the A100.

- `max_new_tokens=256`  
  Limits how many tokens the model can generate in response to a prompt. A larger value allows for longer, more detailed outputs.

> 💡 This pipeline simplifies the generation process so you can just call `generator(prompt)` and receive a coherent answer in return.


In [16]:
generator = pipeline(
    "text-generation",
    model=gemma_model,
    tokenizer=tokenizer,
    device_map=0,
    torch_dtype=torch.bfloat16,
    max_new_tokens=256, # Increased max tokens for more detailed responses
)

Device set to use cuda:0


### 📑 Step 2: Text Snippet Retrieval Setup

In this cell, we define a list of short narrative passages or **context snippets** that describe key events, locations, and interactions between characters (Ethan and Fiona). These text entries will later serve as the **knowledge base** for answering questions using semantic search.

- `text_snippets = [...]`  
  This is a Python list that contains multiple text strings. Each string represents a small piece of a story or description.

These snippets will be:
- Embedded using a sentence transformer model.
- Indexed using FAISS for fast similarity search.
- Used as context when answering user questions via a large language model.

> 📚 This is a crucial part of the RAG (Retrieval-Augmented Generation) setup, where relevant knowledge is retrieved from this list and passed as input to the language model for grounded, context-aware answers.


In [17]:
# 2. Text Snippet Retrieval Setup
text_snippets = [
    "Fiona thanked Ethan for his unwavering support and promised to cherish their friendship.",
    "As they ventured deeper into the forest, they encountered a wide array of obstacles.",
    "Ethan and Fiona crossed treacherous ravines using rickety bridges, relying on each other's strength.",
    "Overwhelmed with joy, Fiona thanked Ethan and disappeared into the embrace of her family.",
    "Ethan returned to his cottage, heart full of memories and a smile brighter than ever before.",
    "The forest was dark and mysterious, filled with ancient trees and hidden paths.",
    "Ethan always carried a map and compass, ensuring they never lost their way.",
]

### 🔍 Step 3: Enhanced Retrieval Mechanism — Semantic Search with FAISS

This section sets up the **semantic embedding** and **vector search index** needed to perform efficient and meaningful retrieval of relevant text snippets based on a user's query.

---

- `embedding_model = SentenceTransformer("all-MiniLM-L6-v2")`  
  Loads a lightweight, high-performance sentence embedding model from the Sentence Transformers library. This model converts sentences into dense numerical vectors (embeddings) that capture their semantic meaning.

- `embeddings_text_snippets = embedding_model.encode(text_snippets)`  
  Generates vector embeddings for each of the predefined text snippets. These embeddings will later be compared to the query embedding to find the most relevant snippet.

---

### ⚙️ FAISS Index Creation

- `dimension = embeddings_text_snippets.shape[1]`  
  Extracts the dimensionality of each embedding vector (e.g., 384), which is required to initialize the FAISS index correctly.

- `index = faiss.IndexFlatL2(dimension)`  
  Initializes a **FAISS index** that uses L2 distance (Euclidean distance) to compare vectors. This allows for fast and efficient similarity search between embeddings.

- `index.add(embeddings_text_snippets.astype(np.float32))`  
  Adds all the text snippet embeddings to the FAISS index after converting them to `float32`, which is the required input format for FAISS.

> ⚡ This enables real-time semantic search, where a user’s question can be matched to the most semantically similar snippet — even if they use different words.


In [18]:
# 3. Enhanced Retrieval Mechanism: Semantic Search with FAISS
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings_text_snippets = embedding_model.encode(text_snippets)

# FAISS Index Creation
dimension = embeddings_text_snippets.shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dimension)  # L2 distance (Euclidean)
index.add(embeddings_text_snippets.astype(np.float32))  # FAISS requires float32

### 🧠 Step 4: Retrieval Function (Semantic Search)

This function takes a user query and returns the **most semantically similar snippet** from the previously indexed text corpus using **FAISS-based nearest neighbor search**.

---

- `def retrieve_snippet(query, k=1):`  
  Defines a Python function that accepts a query string and retrieves `k` most similar snippets. By default, `k=1`, meaning it returns only the top match.

- `query_embedded = embedding_model.encode([query]).astype(np.float32)`  
  Converts the query string into an embedding vector using the same sentence embedding model used for the snippets. FAISS requires all vectors to be in `float32`, so the type is cast accordingly.

- `D, I = index.search(query_embedded, k)`  
  Searches the FAISS index to find the `k` most similar embeddings to the query.  
  - `D`: distances (lower = more similar)  
  - `I`: indices of the most similar snippets in the original list

- `retrieved_indices = I[0]`  
  Extracts the list of top-k indices from the FAISS result. Since only one query is being processed, we access the first (and only) row of `I`.

- `retrieved_texts = [text_snippets[i] for i in retrieved_indices]`  
  Uses the retrieved indices to extract the corresponding text snippets from the original list.

- `return retrieved_texts[0]`  
  Returns only the **most relevant snippet**. This snippet will later be used as context for the language model during text generation.

> 💡 This function powers the semantic retrieval part of RAG — ensuring the model responds using real context instead of hallucinating answers.


In [19]:
# 4. Retrieval Function (Semantic Search)
def retrieve_snippet(query, k=1):  # k is the number of snippets to retrieve
    query_embedded = embedding_model.encode([query]).astype(np.float32)
    D, I = index.search(query_embedded, k)  # D: distances, I: indices
    retrieved_indices = I[0]
    retrieved_texts = [text_snippets[i] for i in retrieved_indices]
    return retrieved_texts[0]  # Return only the top snippet


In [20]:
# 5. Create a function to generate the answer based on the retrieved snippet and query
def ask_query(query):
    retrieved_text = retrieve_snippet(query)

    # Step 1: Construct chat messages as a list of roles/content
    chat = [
        {
            "role": "system",
            "content": "You are a helpful AI assistant. Answer the question based on the context provided.",
        },
        {
            "role": "user",
            "content": f"""Context:
{retrieved_text}

Question: {query}""",
        },
    ]

    # Step 2: Use tokenizer's chat template to format this
    prompt_ids = tokenizer.apply_chat_template(
        chat,
        tokenize=True,
        add_generation_prompt=True,  # add assistant tag to begin model generation
        return_tensors="pt"
    ).to(gemma_model.device)

    # Step 3: Generate using the raw model
    output = gemma_model.generate(prompt_ids, max_new_tokens=128)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f"Query: {query}")
    print(f"Context: {retrieved_text}")
    print(f"Answer: {response}")
    print("-" * 40)


### 🗣️ Step 6: Ask Questions

This block runs a series of **user-defined natural language queries** through the full Retrieval-Augmented Generation (RAG) pipeline, using the `ask_query()` function. For each question, the pipeline:

1. **Finds the most semantically similar snippet** using FAISS-based search.
2. **Constructs a prompt** that includes the retrieved snippet as context.
3. **Generates an answer** using the Gemma language model.

---

- `query1 = "Why did Fiona thank Ethan?"`  
  A straightforward question to test if the model can connect Fiona’s gratitude to Ethan’s support.  
  → Passed to `ask_query(query1)` to fetch the answer.

- `query2 = "What challenges did Ethan and Fiona face in the forest?"`  
  A more complex question that probes the model’s understanding of events and obstacles.  
  → Answer will depend on the forest-related snippets.

- `query3 = "What tools did Ethan use to navigate?"`  
  A factual retrieval question. The model should extract and summarize tools like a map or compass.

- `query4 = "Describe the forest."`  
  An open-ended descriptive query that should trigger a more vivid narrative response based on stored context.

> 🧠 These queries showcase how the system can handle **factual, contextual, and descriptive questions** using real context — avoiding hallucinated answers.


In [21]:
# 6. Ask Questions
query1 = "Why did Fiona thank Ethan?"
ask_query(query1)

query2 = "What challenges did Ethan and Fiona face in the forest?"
ask_query(query2)

query3 = "What tools did Ethan use to navigate?"
ask_query(query3)

query4 = "Describe the forest."
ask_query(query4)

Query: Why did Fiona thank Ethan?
Context: Fiona thanked Ethan for his unwavering support and promised to cherish their friendship.
Answer: user
You are a helpful AI assistant. Answer the question based on the context provided.

Context:
Fiona thanked Ethan for his unwavering support and promised to cherish their friendship.

Question: Why did Fiona thank Ethan?
model
Fiona thanked Ethan for his unwavering support. 

----------------------------------------
Query: What challenges did Ethan and Fiona face in the forest?
Context: Ethan and Fiona crossed treacherous ravines using rickety bridges, relying on each other's strength.
Answer: user
You are a helpful AI assistant. Answer the question based on the context provided.

Context:
Ethan and Fiona crossed treacherous ravines using rickety bridges, relying on each other's strength.

Question: What challenges did Ethan and Fiona face in the forest?
model
Based on the context, Ethan and Fiona faced the challenge of crossing treacherous rav